Part 3: Applied Machine Learning

In [0]:
# Import necessary packages and modules

import time
import pandas as pd
import numpy as np
import datetime as datetime
from datetime import timedelta
import databricks.koalas as ks
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.ml.stat import Correlation
import pyspark.sql.functions as f
from pyspark.sql.functions import *

Task 1
With this fitted model we can predict on the training data and convince ourselves that the model works.

In [0]:
import numpy as np 
import pandas as pd 
from sklearn.linear_model import LogisticRegression
df = pd.read_csv("/test/iris.csv", names = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])
df.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
# Separate features from class. 
array = df.values 
X = array[:,0:4] 
y = array[:,4]
# Fit Logistic Regression classifier. 
logreg = LogisticRegression(C=1e5) 
logreg.fit(X, y)

Out[147]: LogisticRegression(C=100000.0, class_weight=None, dual=False,
 fit_intercept=True, intercept_scaling=1, l1_ratio=None,
 max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
 random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
 warm_start=False)

In [0]:
# Predict on training data. Seems to work. # 5.1     3.5     1.4     0.2     Iris-setosa # 6.2     3.4     5.4     2.3     Iris-virginica
print(logreg.predict([[5.1, 3.5, 1.4, 0.2]])) 
print(logreg.predict([[6.2, 3.4, 5.4, 2.3]]))

['Iris-setosa']
['Iris-virginica']

In [0]:
pred_data = spark.createDataFrame(    [(5.1, 3.5, 1.4, 0.2),      (6.2, 3.4, 5.4, 2.3)],    ["sepal_length", "sepal_width", "petal_length", "petal_width"])

In [0]:
pred_data.dtypes

Out[229]: [('sepal_length', 'double'),
 ('sepal_width', 'double'),
 ('petal_length', 'double'),
 ('petal_width', 'double')]

Task 2
Implement a Spark version of the above Python code and demonstrate that you can correctly predict on the training data, similar to what was done in the preceding section.


In [0]:
df_spk = spark.read.csv("/test/iris.csv")
df_spk.head()
#df = sqlContext.read.csv("/path/to/your.csv")

Out[221]: Row(_c0='5.1', _c1='3.5', _c2='1.4', _c3='0.2', _c4='Iris-setosa')

In [0]:
from pyspark.sql.functions import col

df_spk = df_spk.select(col("_c0").alias("sepal_length"), col("_c1").alias("sepal_width"), col("_c2").alias("petal_length"), col("_c3").alias("petal_width"), col("_c4").alias("class"))
df_spk.show()

+------------+-----------+------------+-----------+-----------+
sepal_length|sepal_width|petal_length|petal_width| class|
+------------+-----------+------------+-----------+-----------+
 5.1| 3.5| 1.4| 0.2|Iris-setosa|
 4.9| 3.0| 1.4| 0.2|Iris-setosa|
 4.7| 3.2| 1.3| 0.2|Iris-setosa|
 4.6| 3.1| 1.5| 0.2|Iris-setosa|
 5.0| 3.6| 1.4| 0.2|Iris-setosa|
 5.4| 3.9| 1.7| 0.4|Iris-setosa|
 4.6| 3.4| 1.4| 0.3|Iris-setosa|
 5.0| 3.4| 1.5| 0.2|Iris-setosa|
 4.4| 2.9| 1.4| 0.2|Iris-setosa|
 4.9| 3.1| 1.5| 0.1|Iris-setosa|
 5.4| 3.7| 1.5| 0.2|Iris-setosa|
 4.8| 3.4| 1.6| 0.2|Iris-setosa|
 4.8| 3.0| 1.4| 0.1|Iris-setosa|
 4.3| 3.0| 1.1| 0.1|Iris-setosa|
 5.8| 4.0| 1.2| 0.2|Iris-setosa|
 5.7| 4.4| 1.5| 0.4|Iris-setosa|
 5.4| 3.9| 1.3| 0.4|Iris-setosa|
 5.1| 3.5| 1.4| 0.3|Iris-setosa|
 5.7| 3.8| 1.7| 0.3|Iris-setosa|
 5.1| 3.8| 1.5| 0.3|Iris-setosa|
+------------+-----------+------------+-----------+-----------+
only showing top 20 rows

In [0]:
feature_cols = df_spk.columns[:-1]

In [0]:
#df_spk.dtypes
df_spk = df_spk.select(df_spk.sepal_length.cast("float"),
df_spk.sepal_width.cast("float"),
df_spk.petal_length.cast("float"),
df_spk.petal_width.cast("float"),
df_spk['class'] )                    

In [0]:
# vectorize all numerical columns into a single feature column
feature_cols = df_spk.columns[:-1]
assembler = pyspark.ml.feature.VectorAssembler(inputCols=feature_cols, outputCol='features')
data = assembler.transform(df_spk)
test_data = assembler.transform(pred_data)

In [0]:

# convert text labels into indices
data = data.select(['features', 'class'])
label_indexer = pyspark.ml.feature.StringIndexer(inputCol='class', outputCol='label').fit(data)
data = label_indexer.transform(data)

In [0]:
data_with_original_label=data

In [0]:
# only select the features and label column
data = data.select(['features', 'label'])
data.show(10)

+--------------------+-----+
 features|label|
+--------------------+-----+
[5.09999990463256...| 0.0|
[4.90000009536743...| 0.0|
[4.69999980926513...| 0.0|
[4.59999990463256...| 0.0|
[5.0,3.5999999046...| 0.0|
[5.40000009536743...| 0.0|
[4.59999990463256...| 0.0|
[5.0,3.4000000953...| 0.0|
[4.40000009536743...| 0.0|
[4.90000009536743...| 0.0|
+--------------------+-----+
only showing top 10 rows

In [0]:
# change regularization rate and you will likely get a different accuracy
reg=1e5
lr = pyspark.ml.classification.LogisticRegression(regParam=reg)
model = lr.fit(data)

In [0]:
# predict on the test set
prediction = model.transform(test_data)
print("Prediction")
prediction.show(10)

Prediction
+------------+-----------+------------+-----------+-----------------+--------------------+--------------------+----------+
sepal_length|sepal_width|petal_length|petal_width| features| rawPrediction| probability|prediction|
+------------+-----------+------------+-----------+-----------------+--------------------+--------------------+----------+
 5.1| 3.5| 1.4| 0.2|[5.1,3.5,1.4,0.2]|[1.71539473284853...|[0.33333905133851...| 0.0|
 6.2| 3.4| 5.4| 2.3|[6.2,3.4,5.4,2.3]|[-9.2706280312180...|[0.33333024312886...| 2.0|
+------------+-----------+------------+-----------+-----------------+--------------------+--------------------+----------+

In [0]:
data_with_original_label.select(['class','label']).distinct().show()

+---------------+-----+
 class|label|
+---------------+-----+
 Iris-setosa| 0.0|
 Iris-virginica| 2.0|
Iris-versicolor| 1.0|
+---------------+-----+